In [41]:
#coding:utf-8

#import library
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime as dt
import datetime
import configparser
import requests
import subprocess
import os

#variable
config = configparser.ConfigParser()
config.read('../config/kyotei.config')

from_date = config.get('date', 'from_date')
to_date = config.get('date', 'to_date')

download_url = config.get('file', 'download_url')
file_dir = config.get('file', 'file_dir')
file_name_race_result = config.get('file', 'file_name_race_result')
file_name_lzh = ''
file_name_txt = ''

db_connect_info = config.get('db', 'db_connect_info')
db_host = config.get('db', 'db_host')
db_port = config.get('db', 'db_port')
db_dbname = config.get('db', 'db_dbname')
db_user = config.get('db', 'db_user')
db_password = config.get('db', 'db_password')

#-----------------------------
#function for download and clean raceinfo file
#-----------------------------
def DownloadFile():
    global file_name_lzh
    global file_name_txt
    
    file_name_lzh = 'k' + race_date[2:8] + '.lzh'
    file_name_txt = 'k' + race_date[2:8] + '.txt'

    #download file
    url = download_url + race_date[0:6] +  '/'+ file_name_lzh
    response = requests.get(url)
    with open(file_dir + file_name_lzh, 'wb') as saveFile:
        saveFile.write(response.content)
    
    #lha file
    cmd = "lha ew=%s %s" % (file_dir, file_dir + file_name_lzh)
    returncode = subprocess.call(cmd, shell=True)
    
    #encoding file
    cmd = "iconv -f SHIFT_JIS -t UTF-8 %s > %s" % (file_dir + file_name_txt, file_dir + file_name_race_result)
    returncode = subprocess.call(cmd, shell=True)

def CleanFile():
    #remove download_file
    cmd = "rm %s" % (file_dir + 'k*')
    returncode = subprocess.call(cmd, shell=True)

    #remove encoding file
    cmd = "rm %s" % (file_dir + file_name_race_result)
    returncode = subprocess.call(cmd, shell=True)

#-----------------------------
#function for set raceinfo
#-----------------------------
def SetRaceResult():
    race_result = pd.DataFrame(columns=['race_date', 'kaijo', 'race_no', 'distance', 'weather', 'wind_direction', 'wind_volume', 'wave', 'rank', 'entry_order', 'toroku_no', 'motor_no', 'boat_no', 'tenji'])
    race_result_odds = pd.DataFrame(columns=['race_date', 'kaijo', 'race_no', 'tansyo', 'fukusyo_1', 'fukusyo_2', 'rentan_2', 'renpuku_2', 'rentan_3', 'renpuku_3'])
    abc=1
    with open(file_dir + file_name_race_result, 'r') as f:
        for line in f:
            '''
            set race_result variable
                date
                kaijo
                race_no
                distance
                weather
                wind_direction
                wind_volume
                wave
                rank
                entry_order
                toroku_no
                motor_no
                boat_no
                tenji
                
            set race_result_odds variable
                tansyo
                fukusyo_1
                fukusyo_2
                rentan_2
                renpuku_2
                rentan_3
                renpuku_3
            ''' 
            print(str(abc) + ' ' + str(len(line)))
            abc += 1
            if line.find('BGN') > -1:
                kaijo = line[0:3]
            
            if line[4:5]=='R':
                race_no = int(line[2:4])
                distance = int(line[line.find('H')+1:line.find('H')+5])
                weather = line[line.find('H')+7:line.find('H')+10].strip()
                wind_direction = line[line.find('H')+15:line.find('H')+18].strip()
                wind_volume = int(line[line.find('H')+19:line.find('H')+20])
                wave = int(line[line.find('H')+25:line.find('H')+28])
            
            if len(line) == 59 and line[2:3] == '0':
                rank = int(line[2:4])
                entry_order = int(line[6:7])
                toroku_no = int(line[8:12])
                motor_no = int(line[22:24])
                boat_no = int(line[27:29])
                tenji = float(line[30:35])
                
                #set race_result dataframe 
                race_result_tmp = pd.DataFrame([[race_date, kaijo, race_no, distance, weather, wind_direction, wind_volume, wave, rank, entry_order, toroku_no, motor_no, boat_no, tenji]], columns=['race_date', 'kaijo', 'race_no', 'distance', 'weather', 'wind_direction', 'wind_volume', 'wave', 'rank', 'entry_order', 'toroku_no', 'motor_no', 'boat_no', 'tenji'])
                race_result = race_result.append(race_result_tmp)
                
            if line.find('単勝') > -1:
                if line.find('特払') > -1:
                    tansyo = int(line[23:30])
                else:
                    if line[20:30].strip().isdigit() == True:
                        tansyo = int(line[20:30])
                    else:
                        tansyo = 100
            if line.find('複勝') > -1:
                if line[20:30].strip().isdigit() == True:
                    fukusyo_1 = int(line[20:30])
                else:
                    fukusyo_1 = 100
                if line[36:45].strip().isdigit() == True:
                    fukusyo_2 = int(line[36:45])
                else:
                    fukusyo_2 = 100
            if line.find('２連単') > -1 and line.find('人気') > -1:
                rentan_2 = int(line[20:30])
            if line.find('２連複') > -1 and line.find('人気') > -1:
                renpuku_2 = int(line[20:30])
            if line.find('３連単') > -1 and line.find('人気') > -1:
                rentan_3 = int(line[20:30])
            if line.find('３連複') > -1 and line.find('人気') > -1:
                renpuku_3 = int(line[20:30])
                
                #set race_result_odds dataframe
                race_result_odds_tmp = pd.DataFrame([[race_date, kaijo, race_no, tansyo, fukusyo_1, fukusyo_2, rentan_2, renpuku_2, rentan_3, renpuku_3]], columns=['race_date', 'kaijo', 'race_no', 'tansyo', 'fukusyo_1', 'fukusyo_2', 'rentan_2', 'renpuku_2', 'rentan_3', 'renpuku_3'])
                race_result_odds = race_result_odds.append(race_result_odds_tmp)
                
    return race_result, race_result_odds
 
#-----------------------------
#functino for postgresql
#-----------------------------    
def Pands2Postgre(race_result, race_result_odds):
    engine=create_engine(db_connect_info)

    race_result.to_sql("wk_race_result", engine, if_exists="replace", index=False)
    race_result_odds.to_sql("wk_race_result_odds", engine, if_exists="replace", index=False)
    
def ExecuteSql(sqltext):
    conn = psycopg2.connect("host=" + db_host + " port=" + db_port + " dbname=" + db_dbname + " user=" + db_user + " password=" + db_password)
    cur = conn.cursor()

    with open(sqltext,'r') as f:
        delete_race_result = f.read()
    
        cur.execute(delete_race_result)
        
        conn.commit()
        cur.close()
        conn.close()
    

#-----------------------------
#function for main
#-----------------------------
def main():
    
    #race result to pandas
    race_result, race_result_odds = SetRaceResult()
    
    #from pandas to postgresql (copy)
    Pands2Postgre(race_result, race_result_odds)
    
    #insert race_result
    ExecuteSql('../sql/delete_race_result.sql')
    ExecuteSql('../sql/insert_race_result.sql')
    
    #insert race_result_odds
    ExecuteSql('../sql/delete_race_result_odds.sql')
    ExecuteSql('../sql/insert_race_result_odds.sql')
    
    #clean file
    CleanFile()
    
    return race_result, race_result_odds

    
#-----------------------------
#execute
#-----------------------------
if __name__ == '__main__':
    
    while dt.strptime(from_date, '%Y%m%d').strftime('%Y%m%d') <= dt.strptime(to_date, '%Y%m%d').strftime('%Y%m%d'):
        
        race_date = from_date
        
        DownloadFile()
        print(file_dir + file_name_txt)
        
        if os.path.isfile(file_dir + file_name_txt) == False:
            from_date = (dt.strptime(from_date, '%Y%m%d') + datetime.timedelta(days=1)).strftime('%Y%m%d')
            print(str(race_date) + ' pass')
            continue
        
        main()
        
        from_date = (dt.strptime(from_date, '%Y%m%d') + datetime.timedelta(days=1)).strftime('%Y%m%d')
        
        print(str(race_date) + ' complete')
    
    print('ALL complete')
    

../file/k171003.txt
1 7
2 7
3 40
4 1
5 41
6 1
7 36
8 1
9 66
10 1
11 41
12 1
13 59
14 76
15 76
16 76
17 76
18 76
19 76
20 76
21 76
22 76
23 76
24 76
25 76
26 1
27 1
28 1
29 66
30 56
31 80
32 59
33 59
34 59
35 59
36 59
37 59
38 1
39 32
40 48
41 40
42 40
43 40
44 43
45 43
46 40
47 40
48 1
49 1
50 66
51 56
52 80
53 59
54 59
55 59
56 59
57 59
58 59
59 1
60 32
61 48
62 40
63 40
64 40
65 43
66 43
67 40
68 40
69 1
70 1
71 66
72 56
73 80
74 59
75 59
76 59
77 59
78 59
79 59
80 1
81 32
82 48
83 40
84 40
85 40
86 43
87 43
88 40
89 40
90 1
91 1
92 66
93 56
94 80
95 59
96 59
97 59
98 59
99 59
100 59
101 1
102 32
103 48
104 40
105 40
106 40
107 43
108 43
109 40
110 40
111 1
112 1
113 66
114 56
115 80
116 59
117 59
118 59
119 59
120 59
121 59
122 1
123 32
124 48
125 40
126 40
127 40
128 43
129 43
130 40
131 40
132 1
133 1
134 66
135 56
136 80
137 59
138 59
139 59
140 59
141 59
142 59
143 1
144 32
145 48
146 40
147 40
148 40
149 43
150 43
151 40
152 40
153 1
154 1
155 66
156 56
157 80
158 59
159 59
160

In [37]:
a='   9R       静波まつり選                 H1800m  雨　  風  南　　 1m  波　  2cm'
a='  10R       特別選抜Ｂ戦                 H1800m  雨　  風  南西　 4m  波　  2cm'

a[a.find('H')+15:a.find('H')+18]
a[a.find('H')+19:a.find('H')+20]
a[a.find('H')+25:a.find('H')+28]

#wind_direction = line[line.find('風')+1:line.find('風')+5].strip()
#wind_volume = int(line[line.find('風')+6:line.find('風')+8])
#wave = int(line[line.find('波')+2:line.find('波')+5])


'  2'